In [3]:
print('test')

test


In [ ]:
##### DEPLOY
#####

In [4]:
from time import gmtime, strftime

In [5]:
# Importing the necessary SageMaker modules for model deployment.
import sagemaker
from sagemaker import ModelPackage

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml


In [6]:
# Set deploy variables
model_package_arn = 'arn:aws:sagemaker:us-east-1:828238096174:model-package/apg-model-pkg-heart/1'
role = sagemaker.get_execution_role()
sagemaker_session =  sagemaker.session.Session()

In [7]:
# Initializing a SageMaker ModelPackage object with the specified parameters.
model = ModelPackage(role=role, 
                     model_package_arn=model_package_arn, 
                     sagemaker_session=sagemaker_session)

In [8]:
# Deploying the SageMaker model to an endpoint with the specified instance configuration
predictor = model.deploy(initial_instance_count=1, instance_type='ml.m5.large')

-------!

In [ ]:
##### PREDICT
#####

In [9]:
import pandas as pd
import numpy as np
import pathlib
import boto3

In [10]:
#Creating a new directory for test data processing, ensuring the parent directories are also created.
pathlib.Path(f"/opt/ml/processing/test").mkdir(parents=True, exist_ok=True)

In [22]:
# Set predict variables
f_bucket = "sagemaker-us-east-1-828238096174"
f_bucketpath = "apg-pipe-heart-p-ewf8t7lvhivm/1j55p30wb5xc/APG-Heart-Process/output/test/test.csv"
f_syspath= "/opt/ml/processing/test/test.csv"
test_path = "/opt/ml/processing/test/test.csv"
#endpoint_name = "apg-model-pkg-heart-endpoint"
endpoint_name = "apg-model-pkg-heart-2024-10-28-22-38-37-317"

In [23]:
#Download the test CSV file
s3 = boto3.resource("s3")
s3.Bucket(f_bucket).download_file(f_bucketpath, f_syspath)

In [24]:
#Reads a CSV file, drops the target column, and cast as string
df = pd.read_csv(test_path, header=None)
df.drop(df.columns[0], axis=1, inplace=True)
data_input = df.to_csv(index=False)

In [25]:
print(data_input[:250])

1,2,3,4,5,6,7,8,9,10,11,12,13,14
40,1,2,0,95.0,252.0,0,2,144.0,0,0.0,2,0.0,0
54,1,1,1,160.0,195.0,0,2,130.0,0,1.0,2,0.0,0
38,0,2,0,110.0,252.0,0,1,156.0,0,0.0,1,0.0,1
63,1,2,0,150.0,250.0,0,2,154.0,0,3.7,2,0.0,0
52,1,1,0,140.0,266.0,0,1,134.0,1,2.0,1


In [26]:
#Creates SageMaker runtime client for interacting with the inference API.
sagemaker_runtime_client = boto3.client('sagemaker-runtime')

In [28]:
#Invokes the specified SageMaker endpoint with the provided CSV data and returns the inference response.
response = sagemaker_runtime_client.invoke_endpoint(
    EndpointName=endpoint_name,
    Body=data_input,
    ContentType='text/csv'
    #Accept='application/json'
)

In [29]:
# Print Inference result
print(response["Body"].read())

b'1.0\n2.0\n2.0\n2.0\n0.0\n2.0\n2.0\n1.0\n2.0\n2.0\n2.0\n2.0\n2.0\n2.0\n2.0\n2.0\n2.0\n2.0\n2.0\n2.0\n2.0\n2.0\n1.0\n2.0\n0.0\n1.0\n2.0\n2.0\n1.0\n2.0\n2.0\n2.0\n2.0\n2.0\n2.0\n2.0\n1.0\n2.0\n0.0\n1.0\n0.0\n2.0\n2.0\n2.0\n1.0\n2.0\n2.0\n2.0\n2.0\n2.0\n2.0\n2.0\n2.0\n2.0\n1.0\n2.0\n2.0\n2.0\n2.0\n2.0\n2.0\n2.0\n1.0\n2.0\n0.0\n2.0\n2.0\n2.0\n2.0\n2.0\n0.0\n1.0\n0.0\n2.0\n2.0\n2.0\n1.0\n1.0\n2.0\n0.0\n2.0\n1.0\n2.0\n2.0\n1.0\n2.0\n2.0\n0.0\n2.0\n2.0\n2.0\n2.0\n2.0\n'
